In [ ]:
import numpy as np
import h5py
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.animation as animation
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.stats import gaussian_kde
from ipywidgets import interact, IntSlider
import seaborn as sns
from matplotlib.colors import LogNorm
import os
import src
from time import time
import mplhep as hep
hep.set_style(hep.style.CMS)

#pip install mplhep

In [ ]:
#hep.style.plt_style

## Set inputs

Optimal inputs for efficiency analysis:
- n_bins = 1000
- q_max = 100000

In [ ]:
# chunk size
chunk_size = 1000

# make a histogram with n bins
n_bins = 40

# charge maximum cut in [e-]
q_max = 100000

# tof maximum cut in [ns]
tof_max = 50

#tof scaling
tof_scaling = 1#1e-9

#q scaling
q_scaling = 1  # 1e9/3.67 # from GeV to eV, from eV to e (3.6eV / e- in Si)

#q threshold
q_threshold = 1000

# window size: extend by this many in both directions
shift_n_times = 0

# tof offset in [ns]
shift_offset = 25

# print infos during run
verbose = False

# width of tornado mask [ns]
mask_xwidth = 25

#shift mask along x axis [ns]
mask_xoffset = 25

#set to -1 to mirror curves against time; stretch mask along x axis
mask_xscale = -1 

# to handle numerical rounding errors
num_precision= 8
epsilon = 1e-8

# remote path to hdf5
remote_path = "../../../sshfs/"

In [ ]:
tof_low_global = -20 - shift_n_times*shift_offset
tof_high_global = tof_max + shift_n_times*shift_offset
q_low_global = 0
q_high_global = q_max

# no epsilon
xedges_global = np.linspace(tof_low_global, tof_high_global, (2*shift_n_times+1)*n_bins+1)
yedges_global = np.linspace(q_low_global, q_high_global, n_bins+1)

## Read hdf5

For BIB the setting __beam 2__ is used. This means that the BIB particles come from the -Z side from -2260 cm. Shower will be produced on the +Z side (D1-4) therefore they will have a larger number of hits. We are interested in the __incoming__ BIB not the shower, so take __disc -4__ that is on the -Z side and is reached first by the BIB particles and therefore has __negative time of flight__ values. For PU samples both disc 4 and -4 are the same as the CMS geometry is symmetric.

In [ ]:
"""
rechit-simhit matching low statistics
"""

hf_bib_halo = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_halo_recsim.h5', 'r')
hf_bib_carbon = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_gas_carbon_recsim.h5', 'r')
hf_bib_oxygen = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_gas_oxygen_recsim.h5', 'r')
hf_bib_hydrogen = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_gas_hydrogen_recsim.h5', 'r')

hf_pu = h5py.File(remote_path+"pu_simulations_fullgeo/hdf5/low_stat_examples/tof_q_PU_recsim.h5", "r")

In [ ]:
dd = [hf_bib_hydrogen]

In [ ]:
dd[0]

In [ ]:
hf_pu_dir = os.listdir(remote_path+"pu_simulations_fullgeo/hdf5")
hf_pu_files = [f for f in hf_pu_dir if ".h5" in f]
hf_pu_list = [] 
for hf_pu_file in hf_pu_files:
    print(hf_pu_file)
    hf_pu_list.append(h5py.File(os.path.join(remote_path+"pu_simulations_fullgeo/hdf5", hf_pu_file), "r"))

In [ ]:
"""
rechit-digi matching low statistics
"""

hf_bib_halo = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_halo.h5', 'r')
hf_bib_carbon = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_gas_carbon.h5', 'r')
hf_bib_oxygen = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_gas_oxygen.h5', 'r')
hf_bib_hydrogen = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/low_stat_examples/tof_q_gas_hydrogen.h5', 'r')

#hf_pu = h5py.File(remote_path+"pu_simulations_fullgeo/hdf5/tof_q_PU_notimewalk.h5", "r")

In [ ]:
"""
rechit-simhit matching full statistics
"""

hf_bib_halo = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/simhit_rechit/tof_q_halo.h5', 'r')
hf_bib_carbon = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/simhit_rechit/tof_q_gas_carbon.h5', 'r')
hf_bib_oxygen = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/simhit_rechit/tof_q_gas_oxygen.h5', 'r')
hf_bib_hydrogen = h5py.File(remote_path+'bib_simulations_fullgeo/hdf5/simhit_rechit/tof_q_gas_hydrogen.h5', 'r')

hf_pu_dir = os.listdir(remote_path+"pu_simulations_fullgeo/hdf5")
hf_pu_files = [f for f in hf_pu_dir if ".h5" in f]
hf_pu_list = [] 
for hf_pu_file in hf_pu_files:
    print(hf_pu_file)
    hf_pu_list.append(h5py.File(os.path.join(remote_path+"pu_simulations_fullgeo/hdf5", hf_pu_file), "r"))

In [ ]:
np.max(bib_dset_, axis=0)

# Read hdf5 files

In [ ]:
disc=4
ring=1

input_data = [hf_pu_list, hf_bib_halo, hf_bib_carbon, hf_bib_oxygen, hf_bib_hydrogen]
pu_dset_, bib_halo_dset_, bib_carbon_dset_, bib_oxygen_dset_, bib_hydrogen_dset_ = src.readH5Data(
    input_data, disc_list=[disc]*len(input_data), ring_list=[ring]*len(input_data), group_name='Tof_q')

bib_dset_ = np.vstack([bib_carbon_dset_, bib_oxygen_dset_, bib_hydrogen_dset_, bib_halo_dset_])

In [ ]:
# cut off high tof and high charge
bib_halo_dset = bib_halo_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_halo_dset_]]
bib_carbon_dset = bib_carbon_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_carbon_dset_]]
bib_oxygen_dset = bib_oxygen_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_oxygen_dset_]]
bib_hydrogen_dset = bib_hydrogen_dset_[[p[0]<tof_max and p[1]<q_max for p in bib_hydrogen_dset_]]
pu_dset  = pu_dset_[[p[0]<tof_max and p[1]<q_max for p in pu_dset_]]

print("Length of original BIB halo dataset: {}".format(len(bib_halo_dset_)))
print("Length of BIB halo dataset after clip: {} ({} %)".format(len(bib_halo_dset),
                            round(100*len(bib_halo_dset)/(len(bib_halo_dset_)+1e-8), 2)))

print("Length of original BIB carbon dataset: {}".format(len(bib_carbon_dset_)))
print("Length of BIB carbon dataset after clip: {} ({} %)".format(len(bib_carbon_dset),
                            round(100*len(bib_carbon_dset)/len(bib_carbon_dset_), 2)))

print("Length of original BIB oxygen dataset: {}".format(len(bib_oxygen_dset_)))
print("Length of BIB oxygen dataset after clip: {} ({} %)".format(len(bib_oxygen_dset),
                            round(100*len(bib_oxygen_dset)/len(bib_oxygen_dset_), 2)))

print("Length of original BIB hydrogen dataset: {}".format(len(bib_hydrogen_dset_)))
print("Length of BIB hydrogen dataset after clip: {} ({} %)".format(len(bib_hydrogen_dset),
                            round(100*len(bib_hydrogen_dset)/len(bib_hydrogen_dset_), 2)))

print("Length of original PU dataset: {}".format(len(pu_dset_)))
print("Length of PU dataset after clip: {} ({} %)".format(len(pu_dset), round(100*len(pu_dset)/len(pu_dset_), 2)))

Specify bin limits manually to be the same for all datasets:
1000 bins
- q: 0-100000 (100/bin)
- tof: -75-75 (0.1/bin)

Minimum rechit charge is 4750 e-.

In [ ]:
# PU
print("pu")
pu_tof_histo, pu_tof_below_histo, pu_tof_above_histo, pu_xedges, pu_yedges = src.binH5Data(pu_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=tof_scaling, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times,shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose,
    xedges=xedges_global, yedges=yedges_global)

# BIB halo
print("halo")
bib_halo_tof_histo, bib_halo_tof_below_histo, bib_halo_tof_above_histo, bib_halo_xedges, bib_halo_yedges = src.binH5Data(bib_halo_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=tof_scaling, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose, 
    xedges=xedges_global, yedges=yedges_global)

# BIB carbon
print("carbon")
bib_carbon_tof_histo, bib_carbon_tof_below_histo, bib_carbon_tof_above_histo, bib_carbon_xedges, bib_carbon_yedges = src.binH5Data(bib_carbon_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=tof_scaling, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose, 
    xedges=xedges_global, yedges=yedges_global)

# BIB oxygen
print("oxygen")
bib_oxygen_tof_histo, bib_oxygen_tof_below_histo, bib_oxygen_tof_above_histo, bib_oxygen_xedges, bib_oxygen_yedges = src.binH5Data(bib_oxygen_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=tof_scaling, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose, 
    xedges=xedges_global, yedges=yedges_global)
    
# BIB hydrogen
print("hydrogen")
bib_hydrogen_tof_histo, bib_hydrogen_tof_below_histo, bib_hydrogen_tof_above_histo, bib_hydrogen_xedges, bib_hydrogen_yedges = src.binH5Data(bib_hydrogen_dset,
    n_bins=n_bins, chunk_size=chunk_size, tof_scaling=tof_scaling, q_scaling=q_scaling,
    q_threshold=q_threshold, shift_n_times=shift_n_times, shift_offset=shift_offset,
    epsilon=epsilon, verbose=verbose,
    xedges=xedges_global, yedges=yedges_global)

#assert np.sum(pu_yedges == bib_yedges) == n_bins+1, "Histogram bins are not the same"

# sum up bib types
bib_dset = np.vstack([bib_carbon_dset, bib_oxygen_dset, bib_hydrogen_dset, bib_halo_dset])
bib_tof_histo = bib_carbon_tof_histo + bib_oxygen_tof_histo + bib_hydrogen_tof_histo + bib_halo_tof_histo
bib_tof_above_histo = bib_carbon_tof_above_histo + bib_oxygen_tof_above_histo + bib_hydrogen_tof_above_histo + bib_halo_tof_above_histo
bib_tof_below_histo = bib_carbon_tof_below_histo + bib_oxygen_tof_below_histo + bib_hydrogen_tof_below_histo + bib_halo_tof_below_histo

In [ ]:
set(y_pu)

In [ ]:
# PU and BIB on same plot as scatterplots
#plt.style.use(hep.style.CMS)
plt.figure(figsize=(10,10))
plt.subplots_adjust(left=0.2)

x_bib = list(map(lambda x: x[0], bib_dset))
y_bib = list(map(lambda x: x[1], bib_dset))
values_bib = np.vstack((x_bib, y_bib))
kernel_bib = gaussian_kde(values_bib)
kde_bib = kernel_bib.evaluate(values_bib)
norm_bib = Normalize(vmin=kde_bib.min(), vmax=kde_bib.max())
colors_bib = cm.ScalarMappable(norm=norm_bib, cmap='jet').to_rgba(kde_bib)
plt.scatter(x_bib, y_bib,
            label="BIB entries: {}".format(len(bib_dset)), s=2, c=colors_bib)


x_pu = list(map(lambda x: x[0], pu_dset))
y_pu = list(map(lambda x: x[1], pu_dset))
values_pu = np.vstack((x_pu, y_pu))
kernel_pu = gaussian_kde(values_pu)
kde_pu = kernel_pu.evaluate(values_pu)
norm_pu = Normalize(vmin=kde_pu.min(), vmax=kde_pu.max())
colors_pu = cm.ScalarMappable(norm=norm_pu, cmap='viridis').to_rgba(kde_pu)
plt.scatter(x_pu, y_pu,
            label="PU entries: {}".format(len(pu_dset)), s=2, c=colors_pu)

lgnd = plt.legend(markerscale=10)
plt.grid()
plt.xlabel("Time of flight [ns]", fontsize=20)
plt.ylabel("Induced charge [e-]", fontsize=20)
plt.title(r"$\bf{Phase-2}$ Simulation Preliminary", fontsize=24, loc="left")

# Just experiment label and <text> such as 'Preliminary' or 'Simulation'
#hep.cms.text("Phase-2 Simulation Preliminary")
plt.ylim(0, 80000)
plt.xlim(-10, 25)
#plt.savefig("scatter_low_stat_rechit_digi_notimewalk_fine.png")

# 2D histos one by one

In [ ]:
plt.style.use(hep.style.CMS)

plt.figure(figsize=(10,10))
plt.imshow(np.ma.masked_where(bib_halo_tof_histo==0, bib_halo_tof_histo), interpolation='nearest', origin='lower',
        extent=[xedges_global[0], xedges_global[-1], yedges_global[0], yedges_global[-1]],
        aspect="auto")
plt.grid()
plt.xlabel("bib_halo_tof [ns]", fontsize=20)
plt.ylabel("Q [e-]", fontsize=20)
#plt.ylim(0, 100000)
#plt.xlim(-30, 60)
plt.title("halo gas: disc {}, ring {}, {} entries".format(
    disc, ring, int(np.sum(bib_halo_tof_histo))), fontsize=20)

plt.subplots_adjust(left=0.15)
plt.colorbar()
#plt.savefig("bib_gas_halo.png")

In [ ]:
plt.style.use(hep.style.CMS)

plt.figure(figsize=(10,10))
plt.imshow(np.ma.masked_where(bib_carbon_tof_histo==0, bib_carbon_tof_histo), interpolation='nearest', origin='lower',
        extent=[xedges_global[0], xedges_global[-1], yedges_global[0], yedges_global[-1]],
        aspect="auto")
plt.grid()
plt.xlabel("bib_carbon_tof [ns]", fontsize=20)
plt.ylabel("Q [e-]", fontsize=20)
#plt.ylim(0, 100000)
#plt.xlim(-30, 60)
plt.title("Carbon gas: disc {}, ring {}, {} entries".format(
    disc, ring, int(np.sum(bib_carbon_tof_histo))), fontsize=20)

plt.subplots_adjust(left=0.15)
plt.colorbar()
#plt.savefig("bib_gas_hydrogen.png")

In [ ]:
#plt.style.use(hep.style.CMS)

plt.figure(figsize=(10,10))
plt.imshow(np.ma.masked_where(pu_tof_histo==0, pu_tof_histo), interpolation='nearest', origin='lower',
        extent=[pu_xedges[0], pu_xedges[-1], pu_yedges[0], pu_yedges[-1]], aspect="auto")
plt.grid()
plt.xlabel("pu_tof [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0, 100000)
plt.title("Pileup: disc {}, Ring {}, {} entries".format(disc, ring, int(np.sum(pu_tof_histo))), fontsize=20)
plt.colorbar()
#plt.savefig("pu_test.png")

# Load timewalk sim data

In [ ]:
#read timewalk sim
data = pd.read_csv("../tornado_plot/timewalk_Qth1000_RD53B.csv", header=0)

# insert a 0th and a n+1th row to facilitate thresholding at 1000 e-
data = pd.concat([pd.DataFrame(data.iloc[0]).T, data]).reset_index(drop=True)

# convert from seconds to nanoseconds
data["Timewalk"] *= 1e9

In [ ]:
data.at[0, "Timewalk"] = data.at[0, "Timewalk"] + mask_xwidth*tof_scaling
data = pd.concat([data, pd.DataFrame(data.iloc[-1]).T]).reset_index(drop=True)
data.at[len(data)-1, "Qel "] = q_high_global # data.at[len(data)-1, "Qel "]*2

#scale mask e.g. mirror in time
data["Timewalk"] = mask_xscale*data["Timewalk"] + mask_xoffset*tof_scaling

#shift curve to get other boundary
data["timewalk_shifted"] = data["Timewalk"] + mask_xwidth*tof_scaling
data

In [ ]:
#plot
fig = plt.figure(figsize=(10, 10))
for i in range(-shift_n_times, shift_n_times+1):
    l1 = "simulation" if i == 0 else ""
    l2 = "shifted by {}ns".format(mask_xwidth) if i == 0 else ""
    plt.plot(data["Timewalk"]+i*shift_offset*tof_scaling, data["Qel "], label=l1, c="b")
    plt.plot(data["timewalk_shifted"]+i*shift_offset*tof_scaling, data["Qel "], label=l2, c="r")
plt.xlabel("Time of flight [ns]")
plt.ylabel("Q [e-]")
plt.legend()
plt.grid()

# Overlay sim curve and bib data

In [ ]:
combined_tof_above_histo = bib_tof_above_histo + pu_tof_above_histo
overlay_xedges = xedges_global
overlay_yedges = yedges_global

In [ ]:
X_mask, Y_mask = np.meshgrid(overlay_xedges, overlay_yedges)

In [ ]:
plt.figure(figsize=(10, 10))
plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_above_histo==0, combined_tof_above_histo))
#plt.ylim(0, .01*q_scaling)
#plt.xlim(8, 13)
plt.grid()

# tornado mask curves
plt.plot(data["Timewalk"], data["Qel "], label="simulation")
plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted", c="r")
plt.legend()
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)

# Coordinate geometry way to calculate area below curve
Need to run this only once to get the mask bound to the binning. <br>
Use 2 separate masks one for BIB one for PU, instead of one composite. <br>
Loop is done afterwards with steps of one bin width.

In [ ]:
verbose=False

In [ ]:
cx_list = []
cy_list = []

cell_mask_pu = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))#*np.nan
cell_mask_bib = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))#*np.nan

# measure elapsed time
start = time()

# loop goes from 0 to nbins, starts from bottom left corner
# loop over rows
for yi in range(n_bins):
    print("Bin row [{}/{}]".format(yi+1, n_bins))
    
    # loop over columns
    for xi in range((2*shift_n_times+1)*n_bins):
        xmin, xmax = round(overlay_xedges[xi], num_precision), round(overlay_xedges[xi+1], num_precision)
        ymin, ymax = round(overlay_yedges[yi], num_precision), round(overlay_yedges[yi+1], num_precision)
        cell_area = round((xmax - xmin)*(ymax - ymin), num_precision)         
        cx = round((xmax + xmin)/2, num_precision)
        cy = round((ymax + ymin)/2, num_precision)
        
        # loop over masks in order: bib (-25ns offset), pu (0ns offset)
        for mask_idx in range(-1, 1):
            
            """
            place these 2 lines outside for loop if one composite mask is used
            """
            area_below_left_curve = []
            area_below_right_curve = []
        
            if verbose:
                print("working on mask copy {}".format(mask_idx))
                
            #get cell area below all curves
            area_below_left_curve.append(src.getAreaBelowCurve(xmin, xmax, ymin, ymax, cx, cy,
                                        data["Timewalk"]+mask_idx*shift_offset*tof_scaling,
                                        data["Qel "], verbose=verbose))
            area_below_right_curve.append(src.getAreaBelowCurve(xmin, xmax, ymin, ymax, cx, cy,
                                        data["timewalk_shifted"]+mask_idx*shift_offset*tof_scaling,
                                        data["Qel "], verbose=verbose))
            """
            untab all lines below and remove last if if composite mask is used
            """
            # cell mask value is between 0 and 1
            area_between_curves = np.sum(area_below_left_curve) - np.sum(area_below_right_curve)
            cell_mask_value = area_between_curves/cell_area
            
            #fill up mask array with cell mask value
            if cell_mask_value < epsilon:
                cell_mask_value = 0
            else:
                
                #store cells that are between two curves
                cx_list.append(cx)
                cy_list.append(cy)
                
                #numerics correction
                if cell_mask_value > 1:
                    cell_mask_value = 1
                if np.abs(cell_mask_value - 1) < epsilon:
                    cell_mask_value = 1   
      
                #save cell masks into numpy array (start from top left corner)
                if mask_idx==-1: # bib mask
                    cell_mask_bib[yi, xi] = cell_mask_value
                elif mask_idx==0:
                    cell_mask_pu[yi, xi] = cell_mask_value
                else:
                    print("Something's fishy.")
        
end = time()
print("Elapsed time: {} s".format(end-start))

In [ ]:
#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_above_histo==0, combined_tof_above_histo),
                           vmin=1, vmax=np.max(combined_tof_above_histo), norm=LogNorm())

# tornado mask bounding curves
for i in range(-1, 1):
    l1 = "simulation" if i == 0 else ""
    l2 = "shifted by {}ns".format(mask_xwidth) if i == 0 else ""
    plt.plot(data["Timewalk"]+i*shift_offset*tof_scaling, data["Qel "], label=l1, c="b")
    plt.plot(data["timewalk_shifted"]+i*shift_offset*tof_scaling, data["Qel "], label=l2, c="r")

#tornado mask
mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask_bib==-1, cell_mask_bib),
                      alpha=.1, cmap=plt.get_cmap('Greys_r'))

plt.legend(facecolor='white', framealpha=1)
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
#plt.xlim(8, 11)
#cbar = fig.colorbar(data_plot) 
#cbar.set_label('Data count',size=18)
plt.grid()

plt.axvline(data["Timewalk"].iloc[-1], label="mask coordinate", c="g")
plt.title("All hits [ {} hits| 100%]".format(int(np.sum(combined_tof_above_histo))))

# Apply mask to data and produce 3 plots

In [ ]:
cell_mask_pu_list = []
cell_mask_bib_list = []
cell_mask_coord_list = []
mask_coord = data["Timewalk"].iloc[-1]
bin_width = int(xedges_global[-1] - xedges_global[0])/n_bins


mask_coord_bin = int(np.floor(np.abs(mask_coord - xedges_global[0])/bin_width))


# goes from 0 to 199
for i in range(2*n_bins-1):
    
    # get coordinate of tornado mask (original curve loaded from the file, not the 25ns shifted)
    mask_coord_i = mask_coord-bin_width*((n_bins-1)-i)
    cell_mask_coord_list.append(mask_coord_i)
    
    # crop mask (no cropping when i=n_bins-1)
    cell_mask_pu_cropped = cell_mask_pu[:,-(i+1):(2*n_bins-1)-i]
    cell_mask_bib_cropped = cell_mask_bib[:,-(i+1):(2*n_bins-1)-i]

    # pad mask (no padding when i=n_bins-1)
    if i <= n_bins-1:
        cell_mask_pu_pad = np.zeros((n_bins, (n_bins-1)-i))#*np.inf
        cell_mask_pu_padded = np.hstack((cell_mask_pu_cropped, cell_mask_pu_pad))
        
        cell_mask_bib_pad = np.zeros((n_bins, (n_bins-1)-i))#*np.inf
        cell_mask_bib_padded = np.hstack((cell_mask_bib_cropped, cell_mask_bib_pad))
    elif i > n_bins-1:
        cell_mask_pu_pad = np.zeros((n_bins, i-(n_bins-1)))#*np.inf
        cell_mask_pu_padded = np.hstack((cell_mask_pu_pad, cell_mask_pu_cropped))

        cell_mask_bib_pad = np.zeros((n_bins, i-(n_bins-1)))#*np.inf
        cell_mask_bib_padded = np.hstack((cell_mask_bib_pad, cell_mask_bib_cropped))

    print(i, "PU: Crop shape: {} Pad shape: {} Mask coord: {}".format(
        np.shape(cell_mask_pu_cropped), np.shape(cell_mask_pu_pad), mask_coord_i))
 
    print(i, "BIB: Crop shape: {} Pad shape: {} Mask coord: {}".format(
        np.shape(cell_mask_bib_cropped), np.shape(cell_mask_bib_pad), mask_coord_i))
 
    cell_mask_pu_list.append(cell_mask_pu_padded)
    cell_mask_bib_list.append(cell_mask_bib_padded)

In [ ]:
%matplotlib notebook

#animate mask overlay
n_frames = 2*n_bins

#init dictionaries for storing efficiencies
pu_hits_dict = {"alpha":[], "beta":[], "alpha_eff":[], "beta_eff":[]}
bib_hits_dict = {"alpha":[], "beta":[], "alpha_eff":[], "beta_eff":[]}

fig, ax = plt.subplots(figsize=(10,5)) # make figure
plt.subplots_adjust(left=0.2)
ax.grid()
plt.xlabel("Time of flight (ns)", fontsize=16)
plt.ylabel("Charge / {} ns (e-)".format(bin_width), fontsize=16)

"""
plot PU 200 + BIB data
"""
# init, show bib and tof separately with different color mapping ad on log scale (plot just once)
all_mesh = ax.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_above_histo==0, combined_tof_above_histo),
                               vmin=1, vmax=np.max(combined_tof_above_histo),
                               norm=LogNorm(), cmap=plt.get_cmap('viridis'))
#bib_mesh = ax.pcolormesh(X_mask, Y_mask, np.ma.masked_where(bib_tof_above_histo==0, bib_tof_above_histo),
#                              vmin=1, vmax=np.max(combined_tof_above_histo),
#                              norm=LogNorm(), cmap=plt.get_cmap('Greys_r'))
#pu_mesh = ax.pcolormesh(X_mask, Y_mask, np.ma.masked_where(pu_tof_above_histo==0, pu_tof_above_histo),
#                              vmin=1, vmax=np.max(combined_tof_above_histo),
#                              norm=LogNorm(), cmap=plt.get_cmap('viridis'))

"""
colorbar settings
"""
all_cbar = plt.colorbar(all_mesh)
all_cbar.ax.set_ylabel('Entries')
#bib_cbar = plt.colorbar(bib_mesh)
#bib_cbar.ax.set_ylabel('BIB entries')
#pu_cbar = plt.colorbar(pu_mesh)
#pu_cbar.ax.set_ylabel('PU 200 entries')

"""
animation
"""
image = ax.pcolormesh(X_mask, Y_mask, np.zeros_like(combined_tof_above_histo), cmap=plt.get_cmap('Greys_r'),
                alpha=.3, vmin=0, vmax=1)    
mask_boundary, = ax.plot([], [], c="r")

def animate_mask(i):
    # set the data in the axesimage object
    cell_mask_pu_current = cell_mask_pu_list[i]
    cell_mask_bib_current = cell_mask_bib_list[i]
    cell_mask_coord_current = cell_mask_coord_list[i]
    
    # need alpha hits for pu mask and beta hits for bib mask
    pu_alpha_hits = src.mcOverlay(cell_mask_pu_current, pu_tof_above_histo)
    pu_beta_hits = pu_tof_above_histo - pu_alpha_hits
    bib_alpha_hits = src.mcOverlay(cell_mask_bib_current, bib_tof_above_histo)
    bib_beta_hits = bib_tof_above_histo - bib_alpha_hits
    
    pu_hits_dict["alpha"].append(int(np.sum(pu_alpha_hits)))
    bib_hits_dict["alpha"].append(int(np.sum(bib_alpha_hits)))
    pu_hits_dict["beta"].append(int(np.sum(pu_beta_hits)))
    bib_hits_dict["beta"].append(int(np.sum(bib_beta_hits)))
    
    pu_hits_dict["alpha_eff"].append(100*pu_hits_dict["alpha"][-1]/np.sum(pu_tof_above_histo))
    bib_hits_dict["alpha_eff"].append(100*bib_hits_dict["alpha"][-1]/np.sum(bib_tof_above_histo))
    pu_hits_dict["beta_eff"].append(100*pu_hits_dict["beta"][-1]/np.sum(pu_tof_above_histo))
    bib_hits_dict["beta_eff"].append(100*bib_hits_dict["beta"][-1]/np.sum(bib_tof_above_histo))
    
    # update animation
    image.set_array(cell_mask_pu_current+cell_mask_bib_current)
    mask_boundary.set_data(data["Timewalk"]-bin_width*((n_bins-1)-i), data["Qel "])
    
    # update title
    fig_title = "Step {}, Mask coordinate: {} ns\nPU 200: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]\nBIB: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]".format(i,
        round(cell_mask_coord_current, 2),
        pu_hits_dict["alpha"][-1], round(pu_hits_dict["alpha_eff"][-1], 2),
        pu_hits_dict["beta"][-1], round(pu_hits_dict["beta_eff"][-1], 2),
        bib_hits_dict["alpha"][-1], round(bib_hits_dict["alpha_eff"][-1], 2),
        bib_hits_dict["beta"][-1], round(bib_hits_dict["beta_eff"][-1], 2))
    ax.set_title(fig_title, fontsize=12)

    return [image, mask_boundary]

#will make 2*n_bins-1 steps even if the frames is larger
ani = animation.FuncAnimation(fig, animate_mask, frames=n_frames,
                              interval=100, repeat=False)
#ani.save('mask_overlay.gif', writer='imagemagick', fps=30)


In [ ]:
alpha_tot = np.add(pu_hits_dict["alpha"][1:], bib_hits_dict["alpha"][1:])
beta_tot = np.add(pu_hits_dict["beta"][1:], bib_hits_dict["beta"][1:])
alpha_eff_tot = np.add(pu_hits_dict["alpha_eff"][1:], bib_hits_dict["alpha_eff"][1:])
beta_eff_tot = np.add(pu_hits_dict["beta_eff"][1:], bib_hits_dict["beta_eff"][1:])
max_eff = np.argmax(alpha_eff_tot)

In [ ]:
import ROOT as rt
import CMS_lumi, tdrstyle
import array

#set the tdr style
tdrstyle.setTDRStyle()

#change the CMS_lumi variables (see CMS_lumi.py)
CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_sqrtS = "13 TeV" # used with iPeriod = 0, e.g. for simulation-only plots (default is an empty string)

iPos = 11
if( iPos==0 ): CMS_lumi.relPosX = 0.12

H_ref = 600; 
W_ref = 800; 
W = W_ref
H  = H_ref

# 
# Simple example of macro: plot with CMS name and lumi text
#  (this script does not pretend to work in all configurations)
# iPeriod = 1*(0/1 7 TeV) + 2*(0/1 8 TeV)  + 4*(0/1 13 TeV) 
# For instance: 
#               iPeriod = 3 means: 7 TeV + 8 TeV
#               iPeriod = 7 means: 7 TeV + 8 TeV + 13 TeV 
#               iPeriod = 0 means: free form (uses lumi_sqrtS)
# Initiated by: Gautier Hamel de Monchenault (Saclay)
# Translated in Python by: Joshua Hardenbrook (Princeton)
# Updated by:   Dinko Ferencek (Rutgers)
#

iPeriod = 3

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

canvas = rt.TCanvas("c2","c2",50,50,W,H)
canvas.SetFillColor(0)
canvas.SetBorderMode(0)
canvas.SetFrameFillStyle(0)
canvas.SetFrameBorderMode(0)
canvas.SetLeftMargin( L/W )
canvas.SetRightMargin( R/W )
canvas.SetTopMargin( T/H )
canvas.SetBottomMargin( B/H )
canvas.SetTickx(0)
canvas.SetTicky(0)

h =  rt.TH1F("h","h; m_{e^{+}e^{-}} (GeV); Events / 0.5 GeV",80,70,110)

h.SetMaximum(260)

xAxis = h.GetXaxis()
xAxis.SetNdivisions(6,5,0)

yAxis = h.GetYaxis()
yAxis.SetNdivisions(6,5,0)
yAxis.SetTitleOffset(1)

h.Draw()

file = rt.TFile("histo.root","READ")
data = file.Get("data")
MC   = file.Get("MC")

MC.Draw("histsame")
data.Draw("esamex0")

#draw the lumi text on the canvas
CMS_lumi.CMS_lumi(canvas, iPeriod, iPos)

canvas.cd()
canvas.Update()
canvas.RedrawAxis()
frame = canvas.GetFrame()
frame.Draw()


#set the colors and size for the legend
histLineColor = rt.kOrange+7
histFillColor = rt.kOrange-2
markerSize  = 1.0

latex = rt.TLatex()
n_ = 2

x1_l = 0.92
y1_l = 0.60

dx_l = 0.30
dy_l = 0.18
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l

legend =  rt.TPad("legend_0","legend_0",x0_l,y0_l,x1_l, y1_l )
#legend.SetFillColor( rt.kGray )
legend.Draw()
legend.cd()

ar_l = dy_l/dx_l
#gap_ = 0.09/ar_l
gap_ = 1./(n_+1)
bwx_ = 0.12
bwy_ = gap_/1.5

x_l = [1.2*bwx_]
#y_l = [1-(1-0.10)/ar_l]
y_l = [1-gap_]
ex_l = [0]
ey_l = [0.04/ar_l]

#array must be converted 
x_l = array.array("f",x_l)
ex_l = array.array("f",ex_l)
y_l = array.array("f",y_l)
ey_l = array.array("f",ey_l)

gr_l =  rt.TGraphErrors(1, x_l, y_l, ex_l, ey_l)

rt.gStyle.SetEndErrorSize(0)
gr_l.SetMarkerSize(0.9)
gr_l.Draw("0P")

latex.SetTextFont(42)
latex.SetTextAngle(0)
latex.SetTextColor(rt.kBlack)    
latex.SetTextSize(0.25)    
latex.SetTextAlign(12) 

box_ = rt.TBox()
xx_ = x_l[0]
yy_ = y_l[0]
latex.DrawLatex(xx_+1.*bwx_,yy_,"Data")

yy_ -= gap_
box_.SetLineStyle( rt.kSolid )
box_.SetLineWidth( 1 )
# box_.SetLineColor( kBlack )
box_.SetLineColor( histLineColor )
box_.SetFillColor( histFillColor )
box_.DrawBox( xx_-bwx_/2, yy_-bwy_/2, xx_+bwx_/2, yy_+bwy_/2 )
box_.SetFillStyle(0)
box_.DrawBox( xx_-bwx_/2, yy_-bwy_/2, xx_+bwx_/2, yy_+bwy_/2 )
#Draw Z->ee text
latex.DrawLatex(xx_+1.*bwx_,yy_,"Z #rightarrow e^{+}e^{-} (MC)")

#update the canvas to draw the legend
canvas.Update()

raw_input("Press Enter to end")

In [ ]:
%matplotlib inline
plt.ioff()

fix, ax = plt.subplots(2,1)
plt.tight_layout()

ax[0].scatter(cell_mask_coord_list[:n_frames], pu_hits_dict["alpha_eff"][1:], label="PU 200 alpha", c="red", s=3)
ax[1].scatter(cell_mask_coord_list[:n_frames], pu_hits_dict["beta_eff"][1:], label="PU 200 beta", c="red", s=3)
ax[0].scatter(cell_mask_coord_list[:n_frames], bib_hits_dict["alpha_eff"][1:], label="BIB alpha", c="blue", s=3)
ax[1].scatter(cell_mask_coord_list[:n_frames], bib_hits_dict["beta_eff"][1:], label="BIB beta", c="blue", s=3)

ax[0].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)), c="g", linestyle="--", linewidth=1)
ax[1].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)), c="g", linestyle="--", linewidth=1)

hep.cms.text("Phase-2 Simulation Preliminary"+
             " "*47+
             "$\mathrm{\sqrt{s} = 14\,\,TeV}$", loc=0, ax=ax[0], fontsize=10)
hep.cms.text("Phase-2 Simulation Preliminary"+
             " "*47+
             "$\mathrm{\sqrt{s} = 14\,\,TeV}$", loc=0, ax=ax[1], fontsize=10)

ax[0].set_ylim(-5,105)
ax[0].legend(loc="right")
#ax[0].grid()
ax[1].set_ylim(-5,105)
ax[1].legend(loc="right")
#ax[1].grid()

ax[0].set_ylabel("Efficiency / {} ns (%)".format(bin_width), fontsize=16)
ax[1].set_xlabel("Mask coordinate (ns)", fontsize=16)
#ax[0].set_title(r"$\bf{CMS}$ $\it{Phase}$$\it{-2}$ $\it{Simulation}$ $\it{Preliminary}$", fontsize=16, loc="left")


#plt.savefig("efficiencies_test.png")
plt.show()

In [ ]:
plt.ioff()
fix, ax = plt.subplots(2,1)
ax[0].scatter(cell_mask_coord_list[:n_frames], alpha_eff_tot,
              label="Total alpha efficiency", c="k", s=3)
ax[1].scatter(cell_mask_coord_list[:n_frames], beta_eff_tot,
              label="Total beta efficiency", c="k", s=3)

ax[0].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)))
ax[1].axvline(cell_mask_coord_list[max_eff], label="Max. efficiency at {} ns".format(
    round(cell_mask_coord_list[max_eff], 4)))

ax[0].legend()
ax[0].grid()
ax[1].legend()
ax[1].grid()

ax[0].set_ylabel("Efficiency / {} ns (%)".format(bin_width), fontsize=16)
ax[1].set_xlabel("Mask coordinate (ns)", fontsize=16)
ax[0].set_title(r"$\bf{CMS\,\,Phase}\bf{-2}$ Simulation Preliminary", fontsize=16, loc="left")

#plt.savefig("efficiencies_test.png")
plt.show()

In [ ]:
#show plot of data and mask at right coordinate
#max_eff = 28
"""
plot PU 200 + BIB data
"""
fig = plt.figure(figsize=(9,6))
all_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(combined_tof_above_histo==0, combined_tof_above_histo),
                              vmin=1, vmax=np.max(combined_tof_above_histo),
                              norm=LogNorm(), cmap=plt.get_cmap('viridis'))

#bib_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(bib_tof_above_histo==0, bib_tof_above_histo),
#                               vmin=1, vmax=np.max(combined_tof_above_histo),
#                               norm=LogNorm(), cmap=plt.get_cmap('Greys_r'))

#pu_data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(pu_tof_above_histo==0, pu_tof_above_histo),
#                              vmin=1, vmax=np.max(combined_tof_above_histo),
#                              norm=LogNorm(), cmap=plt.get_cmap('viridis'))

plt.axvline(cell_mask_coord_list[max_eff], c="r")

"""
colorbar settings
"""
all_cbar = plt.colorbar(all_data_plot)
all_cbar.ax.set_ylabel('Entries')
#bib_cbar = plt.colorbar(bib_data_plot)
#bib_cbar.ax.set_ylabel('BIB entries')
#pu_cbar = plt.colorbar(pu_data_plot)
#pu_cbar.ax.set_ylabel('PU 200 entries')

"""
tornado mask at max. efficiency
"""
mask = plt.pcolormesh(X_mask, Y_mask, cell_mask_pu_list[max_eff]+cell_mask_bib_list[max_eff],
                      alpha=.5, cmap=plt.get_cmap('Greys_r'))

"""
plot settings
"""
plt.xlabel("Time of flight (ns)", fontsize=16)
plt.ylabel("Charge / {} ns (e-)".format(bin_width), fontsize=16)
#plt.ylim(0,80000)
#plt.xlim(8, 11)
#cbar = fig.colorbar(data_plot) 
#cbar.set_label('Data count',size=18)
plt.grid()

fig_title = "Mask coordinate: {} ns\nPU 200: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]\nBIB: Alpha hits: [{} | {}%] Beta hits: [{} | {}%]".format(
        round(cell_mask_coord_list[max_eff], 2),
        int(pu_hits_dict["alpha"][max_eff+1]), round(100*int(pu_hits_dict["alpha"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(pu_hits_dict["beta"][max_eff+1]), round(100*int(pu_hits_dict["beta"][max_eff+1])/np.sum(pu_tof_above_histo), 2),
        int(bib_hits_dict["alpha"][max_eff+1]), round(100*int(bib_hits_dict["alpha"][max_eff+1])/np.sum(bib_tof_above_histo), 2),
        int(bib_hits_dict["beta"][max_eff+1]), round(100*int(bib_hits_dict["beta"][max_eff+1])/np.sum(bib_tof_above_histo), 2))
plt.title(fig_title)

#plt.title(r"$\bf{CMS\,\,Phase}\bf{-2}$ Simulation Preliminary", fontsize=16, loc="left")
#plt.legend(facecolor='white', framealpha=1)
plt.show()

In [ ]:
aaa = [bool(f) for f in np.random.randint(0,2,10)]
bbb = [bool(f) for f in np.random.randint(0,2,10)]
print(aaa)
print(bbb)
print(aaa or bbb)

In [ ]:
alpha_hits = src.mcOverlay(cell_mask, tof_above_histo)
beta_hits = tof_above_histo - alpha_hits
gamma_hits = tof_below_histo

In [ ]:
#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(alpha_hits==0, alpha_hits),
                           vmin=1, vmax=np.max(combined_tof_above_histo), norm=LogNorm())

# tornado mask bounding curves
#plt.plot(data["Timewalk"], data["Qel "], label="simulation", c="b")
#plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted by 25ns", c="r")

#tornado mask
#mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask==0, cell_mask), alpha=.6, cmap=plt.get_cmap('viridis'))

plt.legend()
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
#plt.xlim(tof_low, tof_high)
cbar = fig.colorbar(data_plot) 
cbar.set_label('Data count',size=18)
plt.grid()

plt.title("Alpha hits [ {} hits| {}%]".format(int(np.sum(alpha_hits)),
                                              round(100*np.sum(alpha_hits)/np.sum(combined_tof_above_histo), 2)))

#plt.savefig("alpha_hits.png")

In [ ]:
#%matplotlib

#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(beta_hits<1e-8, beta_hits),
                           vmin=1, vmax=np.max(combined_tof_above_histo), norm=LogNorm())

# tornado mask bounding curves
#plt.plot(data["Timewalk"], data["Qel "], label="simulation", c="b")
#plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted by 25ns", c="r")

#tornado mask
#mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask==0, cell_mask), alpha=.6, cmap=plt.get_cmap('viridis'))

plt.legend()
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
#plt.xlim(tof_low, tof_high)
cbar = fig.colorbar(data_plot) 
cbar.set_label('Data count',size=18)
plt.grid()

plt.title("Beta hits [ {} hits| {}%]".format(int(np.sum(beta_hits)),
                                             round(100*np.sum(beta_hits)/np.sum(combined_tof_above_histo), 2)))

#plt.savefig("beta_hits.png")

In [ ]:
#%matplotlib

#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(gamma_hits<1e-8, gamma_hits),
                           vmin=1, vmax=np.max(combined_tof_above_histo), norm=LogNorm())

# tornado mask bounding curves
#plt.plot(data["Timewalk"], data["Qel "], label="simulation", c="b")
#plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted by 25ns", c="r")

#tornado mask
#mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask==0, cell_mask), alpha=.6, cmap=plt.get_cmap('viridis'))

plt.legend()
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
#plt.xlim(tof_low, tof_high)
cbar = fig.colorbar(data_plot) 
cbar.set_label('Data count',size=18)
plt.grid()

plt.title("Gamma hits [ {} hits| {}%]".format(int(np.sum(gamma_hits)),
                                              round(100*np.sum(gamma_hits)/np.sum(combined_tof_above_histo), 2)))

#plt.savefig("gamma_hits.png")

In [ ]:
#hf_bib_carbon.close()
#hf_bib_hydrogen.close()
#hf_pu.close()